## Aula 5 - Word Embedding

In [3]:
# Word Embedding
from gensim import *
import spacy
import gensim.downloader

In [4]:
for model_name in list(gensim.downloader.info()["models"].keys()):
    print(model_name)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [5]:
g_news = gensim.downloader.load("word2vec-google-news-300")

[=================================================-] 99.0% 1646.8/1662.8MB downloaded


In [6]:
# Paris está para França, assim como Londres está para... (Inglaterra)
# (Paris - França) + Londres = Inglaterra (positive=["Paris", "Londres"], negative=["França"])

---

### Usando spacy

https://stackoverflow.com/questions/63693463/spacy-lemmatizer-removes-capitalization

In [12]:
# baixando um modelo do spacy já treinada de palavras em inglês
#!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-08-31 19:53:15.651849: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-08-31 19:53:15.652776: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-31 19:53:24.976641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-08-31 19:53:24.977456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-08-31 19:53:24.978228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-08-31 19:53:24.978984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [14]:
# Carregar um pipeline de modelo treinado 'en_core_web_sm' 
nlp = spacy.load("en_core_web_sm")

In [24]:
# modelo nlp com a string
text = nlp("I don't know. Professor of Data Science.")

In [25]:
# Tokenização com lemma, lowercase, sem stopwords e sem pontuação
txt = [token.lemma_.lower() for token in text if not token.is_stop and not token.is_punct]
txt

['know', 'professor', 'data', 'science']

In [ ]:
# Como função
def clear(text):
    global txt = [token.lemma_.lower() for token in text if not token.is_stop and not token.is_punct]
    if len(txt) > 2:
        return " ".join(txt)

In [26]:
# Tag names : https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/
txt2 = [token.tag_ for token in text]
txt2

['PRP', 'VBP', 'RB', 'VB', '.', 'NNP', 'IN', 'NNP', 'NNP', '.']

---

### G_news: modelo já treinado

In [27]:
capital = g_news.most_similar(["Paris", "Italy"], ["France"])
capital

[('Milan', 0.7222141623497009),
 ('Rome', 0.702830970287323),
 ('Palermo_Sicily', 0.5967569947242737),
 ('Italian', 0.5911272764205933),
 ('Tuscany', 0.5632812976837158),
 ('Bologna', 0.5608358383178711),
 ('Sicily', 0.5596384406089783),
 ('Bologna_Italy', 0.5470058917999268),
 ('Berna_Milan', 0.5464026927947998),
 ('Genoa', 0.5308900475502014)]

In [28]:
w = g_news.most_similar('Porsche', topn = 4)
w

[('Audi', 0.760235071182251),
 ('BMW', 0.7272197604179382),
 ('Maserati', 0.7194910645484924),
 ('VW', 0.6961471438407898)]

In [29]:
c = g_news.similarity('Italy', 'Rome')
c

0.5555642

---

In [32]:
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import pandas as pd

In [35]:
df = pd.read_csv('dataset/movies.csv', index_col = 0)

In [37]:
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [43]:
movies_sample = df.sample(frac = 0.2, replace = False)

In [44]:
movies_sample = movies_sample.reset_index(drop = True)
nlp = spacy.load("en_core_web_sm")

In [45]:
def preprocessing(string):
    """
    Preprocessing for english'
    """
    # Manter somente caracteres e números - sem caracteres especiais
    string = re.sub(r"[^a-zA-Z0-9]+", " ", string)
    # Letras minúsculas
    string = string.lower()
    #
    txt = [token.lemma_.lower() for token in string if not token.is_stop and not token.is_punct]
    
    return txt

# melhor usar o antigo com stem e lemma
movies_sample['filtered_words'] = movies_sample["text"].apply(lambda x: nlp(x))
movies_sample['filtered_words'] = movies_sample["filtered_words"].apply(lambda x: preprocessing(x))

In [52]:
movies_sample.head()

,text,label,filtered_words
0,I love this movie. The cast were all terrific ...,1,"(I, love, this, movie, ., The, cast, were, all..."
1,The Lone Ranger & Tonto set out to bring to ju...,1,"(The, Lone, Ranger, &, Tonto, set, out, to, br..."
2,"<br /><br />According to reviewers, the year i...",0,"(<, br, /><br, />According, to, reviewers, ,, ..."
3,Has anyone noticed that James Earl Jones is th...,1,"(Has, anyone, noticed, that, James, Earl, Jone..."
4,This is just a case of a previously worthless ...,0,"(This, is, just, a, case, of, a, previously, w..."


In [53]:
words_list = movies_sample["filtered_words"].to_list()

In [54]:
from gensim.models.phrases import Phraser, Phrases

In [56]:
model = Word2Vec(words_list, min_count = 3, workers = 2)

RuntimeError: you must first build vocabulary before training the model

In [57]:
model.vector_size

NameError: name 'model' is not defined

### Salvando model

In [ ]:
model.save("model_w2v.bin")

In [ ]:
model2 = Word2Vec.load('model_w2v.bin')